In [ ]:
try:
    import BACKEND_LIBRARY_IN  # noqa: F401
except ImportError:
    import nanobind
    import pybind11
else:
    !wget "https://PROJECT_NAME_IN.github.io/releases/pybind11-install.sh" -O "/tmp/pybind11-install.sh" && bash "/tmp/pybind11-install.sh"
    import nanobind  # noqa: F401
    import pybind11  # noqa: F401

In [ ]:
%%file sum.hpp

int sum(int i, int j);

In [ ]:
%%file sum.cpp

int sum(int i, int j) {
    return i + j;
};

In [ ]:
%%file pybind11_sum.cpp
#include <pybind11/pybind11.h>
#include "sum.hpp"

PYBIND11_MODULE(pybind11_sum, m) {
    m.def("sum", &sum);
}

In [ ]:
%%file nanobind_sum.cpp
#include <nanobind/nanobind.h>
#include "sum.hpp"

NB_MODULE(nanobind_sum, m) {
    m.def("sum", &sum);
}

In [ ]:
%%file pybind11_setup.py
import pybind11

from distutils.core import Extension, setup

ext_modules = [
    Extension(
        "pybind11_sum", ["sum.cpp", "pybind11_sum.cpp"],
        include_dirs=[pybind11.get_include()],
        language="c++",
        extra_compile_args=["-std=c++11"]
    ),
]

setup(
    name="pybind11_sum",
    ext_modules=ext_modules
)

In [ ]:
%%file nanobind_CMakeLists.txt

cmake_minimum_required(VERSION 3.15)
project(NANOBIND_SUM)
find_package(Python 3.8 COMPONENTS Interpreter Development.Module REQUIRED)

set(CMAKE_BUILD_TYPE Release CACHE STRING "Choose the type of build." FORCE)
set_property(CACHE CMAKE_BUILD_TYPE PROPERTY STRINGS "Debug" "Release" "MinSizeRel" "RelWithDebInfo")

execute_process(
  COMMAND "${Python_EXECUTABLE}" -m nanobind --cmake_dir
  OUTPUT_STRIP_TRAILING_WHITESPACE OUTPUT_VARIABLE NB_DIR)
list(APPEND CMAKE_PREFIX_PATH "${NB_DIR}")
find_package(nanobind CONFIG REQUIRED)

nanobind_add_module(nanobind_sum sum.cpp nanobind_sum.cpp)

In [ ]:
%%bash

python3 pybind11_setup.py build_ext -i

In [ ]:
%%bash

rm -rf CMake*
ln -s nanobind_CMakeLists.txt CMakeLists.txt
cmake .
VERBOSE=1 make
rm CMakeLists.txt

In [ ]:
import pybind11_sum
assert pybind11_sum.sum(1, 2) == 3

In [ ]:
import nanobind_sum
assert nanobind_sum.sum(1, 2) == 3